In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm


from model.SpeechGenerator import SpeechGenerator
from utils.Config import ConfigSLP, ConfigNAC, ConfigDiTTO
from utils.MLS import MLSDataset
from utils.Trainer import Trainer
from utils.Processing import Processing

from torch.utils.data import DataLoader

In [ ]:
ConfigSLP.display()
ConfigNAC.display()
ConfigDiTTO.display()

In [ ]:
# Processing.remove_metadata_from_audio_folder(ConfigSLP.TRAIN_PATH+"/"+"audio", ConfigSLP.TRAIN_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(ConfigSLP.TEST_PATH+"/"+"audio", ConfigSLP.TEST_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(ConfigSLP.DEV_PATH+"/"+"audio", ConfigSLP.DEV_PATH+"/"+"audio_clean",)

## Speech Generation with DiTTO-TTs and Vocoder

In [ ]:
speech_generator = SpeechGenerator(
    nac_model_path="C:/Cours-Sorbonne/M2/UE_DEEP/AMAL/Projet/src/params/NAC_epoch_20.pth",
    ditto_model_path="C:/Cours-Sorbonne/M2/UE_DEEP/AMAL/Projet/src/params/DiTTO_epoch_20.pth",
    lambda_factor=ConfigNAC.LAMBDA_FACTOR,
    sample_rate=ConfigNAC.SAMPLE_RATE,
)

In [ ]:
speech_generator.generate_speech_from_audio_tensor(
    
)